In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import statsmodels.api as sm

In [ ]:
data = pd.read_excel('creditscore.xlsx')

In [ ]:
data.head()

,BureauInquiries,CreditUsage,TotalCredit,CollectionReports,MissedPayments,HomeOwner,CreditAge,TimeOnJob,Repay,Partition
0,7,0.27,18000,0,2,0,6.4,3,0,t
1,7,0.23,16000,0,1,1,6.7,6,1,t
2,7,0.27,18000,0,1,0,6.4,3,1,t
3,8,0.23,21000,0,0,1,7.1,6,0,t
4,8,0.42,32000,0,2,0,10.1,6,0,t


In [ ]:
data.isnull().sum()

BureauInquiries      0
CreditUsage          0
TotalCredit          0
CollectionReports    0
MissedPayments       0
HomeOwner            0
CreditAge            0
TimeOnJob            0
Repay                0
Partition            0
dtype: int64

In [ ]:
data.duplicated().sum()

214

In [ ]:
data.drop_duplicates(inplace=True) #inplace makes changes in actual data

In [ ]:
data = data.drop('Partition', axis=1)
X = data.drop('Repay', axis=1)
y = data['Repay']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

FULL MODEL- All predictors used

In [ ]:
full_model = LogisticRegression()
full_model.fit(X_train, y_train)

y_pred = full_model.predict(X_test)
full_model_accuracy = accuracy_score(y_test, y_pred)
print(full_model_accuracy)

0.7088948787061995


REDUCED MODEL- Only 4 predictors used

In [ ]:
reduced_model = LogisticRegression()
reduced_model.fit(X_train[['CreditUsage', 'MissedPayments', 'HomeOwner', 'CreditAge']], y_train)

y_pred = reduced_model.predict(X_test[['CreditUsage', 'MissedPayments', 'HomeOwner', 'CreditAge']])
reduced_model_accuracy = accuracy_score(y_test, y_pred)
print(reduced_model_accuracy)

0.7196765498652291


SIGNIFICANT MODEL- Significant predictors used (using statsmodel library)

In [ ]:
# We fit the logistic regression model twice because we first need to identify the significant predictors based on their p-values, and then fit a new logistic regression model using only those significant predictors

X = sm.add_constant(X) # const term added to incl intercept term (intercept term is value of dependent var when all independent var equal to zero)
model = sm.Logit(y, X).fit(disp=False) # fit logistic regression
significant_predictors = list(model.pvalues[model.pvalues < 0.05].index) # get significant predictors
significant_model = sm.Logit(y, X[significant_predictors]).fit(disp=False) # fit significant model with only signnificant predictors

y_pred = significant_model.predict(X_test[significant_predictors])
y_pred_class = [1 if p > 0.5 else 0 for p in y_pred]
signficant_model_accuracy = accuracy_score(y_test, y_pred_class)
print(signficant_model_accuracy)

0.7358490566037735


Accuracy of all three models

In [ ]:
df = pd.DataFrame({'Full model': [full_model_accuracy], 'Reduced Model': [reduced_model_accuracy], 'Signficant Model': [signficant_model_accuracy]})
df

,Full model,Reduced Model,Signficant Model
0,0.708895,0.719677,0.735849


Expressing significant model as a mathematical equation

In [ ]:
#extract coeff
coeff = significant_model.params.apply(lambda x: "{:.5f}".format(x))

# construct eqn
equation = 'p = 1 / (1 + exp(-(' + str(coeff[0]) + ' + '

for i in range(1, len(coeff)):
    equation += str(coeff[i]) + ' * X' + str(i) + ' + '

# remove last '+' and add 3 closing paranthesis
equation = equation[:-3] + ')))'

print(equation)

p = 1 / (1 + exp(-(-0.09569 + -3.67115 * X1 + -1.62960 * X2 + -0.56513 * X3 + 0.12729 * X4 + 0.18555 * X5)))
